In [ ]:
import openpyxl
import pandas as pd
import os
import tkinter as tk
from tkinter import filedialog

In [ ]:
## 파일 경로 받아오는 GUI 호출

root = tk.Tk()
root.withdraw()
dir_path = filedialog.askopenfile(
    parent=root,initialdir=os.getcwd(),title='Please select a file',
    filetypes=(('xls files','*.xls'),('all files','*.*')))

In [ ]:
## 파일 읽어서 Data Frame으로 저장

print("load file: ", dir_path.name)
df = pd.read_excel(dir_path.name, sheet_name=None)

mri_table = pd.DataFrame()
mri_table = pd.concat(df, ignore_index=True)

In [ ]:
# 필요없는 값들 다 지우기

mri_table.drop([0, 1, 2, 3], inplace=True)
mri_table.dropna(axis=1, inplace=True)
mri_table.rename(columns=mri_table.iloc[0],inplace=True)
mri_table.drop([4], inplace=True)

# str --> int 형변환

mri_table["NO"] = pd.to_numeric(mri_table["NO"])
mri_table["등록번호"] = pd.to_numeric(mri_table["등록번호"])

In [18]:
## 나이 필터링 (17세 미만 제외)

from datetime import datetime
from dateutil.relativedelta import relativedelta

pre_mil = mri_table[pd.to_numeric(mri_table["주민번호"].str[7]) < 3]
pre_mil["주민번호"] = "19" + pre_mil["주민번호"]

post_mil = mri_table[pd.to_numeric(mri_table["주민번호"].str[7]) > 2]
post_mil["주민번호"] = "20" + post_mil["주민번호"]

mri_table2 = pd.merge(pre_mil,post_mil, how='outer')
mri_table2 = mri_table2.sort_values(by='NO')
mri_table = mri_table2

mri_table["주민번호"] = mri_table["주민번호"].str[:8]

mri_table["주민번호"] = mri_table["주민번호"].astype('datetime64[ns]')
mri_table = mri_table.rename(columns={'주민번호':'생년월일'})


In [12]:
today = datetime.today()
age = pd.DataFrame()
age["age"] = mri_table['생년월일'].apply(
               lambda x: today.year - x.year - 
               ((today.month, today.day) < (x.month, x.day)) 
               )
mri_table["생년월일"] = age["age"]
mri_table = mri_table.rename(columns={'생년월일':'나이'})

In [ ]:

mri_table.drop(["주민번호","진료일자","병실"],axis=1, inplace=True)

In [ ]:
## remove list 파일 읽어서 Data Frame으로 저장

remove_list_path = "./msk/remove_list.xlsx"
df2 = pd.read_excel(remove_list_path, sheet_name=None, engine='openpyxl')

remove_list_table = pd.DataFrame()
remove_list_table = pd.concat(df2, ignore_index=True)

In [ ]:
## remove list와 일치하는 처방 삭제하기

for i in range(len(remove_list_table)):
    mri_table = mri_table[~mri_table["처방명"].str.contains(remove_list_table["키워드"][i])]

# No 1부터 다시 매기기
mri_table["NO"] = range(1, len(mri_table)+1)